In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
import matplotlib.pyplot as plt
from keras.regularizers import l2
from sklearn.metrics import confusion_matrix, roc_auc_score
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


In [8]:
###GET DATA
df = pd.read_csv('scaled.csv')

In [9]:
labels = df['Outcome']
df.drop(['Outcome'], inplace = True, axis = 1)
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.367647,0.719347,0.581148,0.368182,0.194613,0.500671,0.260974,0.485
1,0.102941,0.434422,0.536885,0.313636,0.104647,0.406781,0.154910,0.200
2,0.473529,0.877638,0.522131,0.184967,0.148408,0.362519,0.278266,0.215
3,0.102941,0.452513,0.536885,0.259091,0.150000,0.426900,0.084202,0.050
4,0.050000,0.669598,0.345082,0.368182,0.228723,0.628092,0.899274,0.230


In [10]:
labels = [1 if i == .95 else 0 for i in labels]

In [11]:
assert(len(labels) == len(df))

In [12]:
x_train, x_test = np.array(df[:623]), np.array(df[623:])
y_train, y_test = np.array(labels[:623]), np.array(labels[623:])

In [13]:
new_df = pd.DataFrame()

In [42]:
X.columns.values

array(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23', '24', '25', '26', '27'], dtype=object)

In [44]:
##CHURN
col_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20',
       '21', '22', '23', '24', '25', '26', '27']
X = pd.read_csv('churn_features.csv')
Y = pd.read_csv('churn_labels.csv')
y = Y['0'].copy()
x = X[col_names].copy()

In [45]:
##CHURN
x_train, x_test = np.array(x[:6000]), np.array(x[6000:])
y_train, y_test = np.array(y[:6000]), np.array(y[6000:])

In [47]:
search_space = {
    'size': [8, 16, 32, 64, 128], 
    'layers': [1, 2, 3],
    'nodes': [32, 64,128, 512, 1024],
    'activation': ['relu', 'tanh'],
    'optimizers': ['sgd', 'adam'],
    'dropout': [0, .3, .4, .5, .6, .7]
}

#initial random probabilities
start_probs = {
    'model_size': [.2, .2, .2, .2, .2], 
    'model_layers': [.33, .34, .33],
    'model_optimizers': [0.5, 0.5],
    'input_nodes': [.2, .2, .2, .2, .2],
    'input_activation': [.5, .5],
    'input_dropout': [.1666, .1666, .167, .1666, .1666, .1666],
    'layer_1_nodes': [.2, .2, .2, .2, .2],
    'layer_1_activation': [.5, .5],
    'layer_1_dropout': [.1666, .1666, .167, .1666, .1666, .1666],
    'layer_2_nodes': [.2, .2, .2, .2, .2],
    'layer_2_activation': [.5, .5],
    'layer_2_dropout': [.1666, .1666, .167, .1666, .1666, .1666],
    'layer_3_nodes': [.2, .2, .2, .2, .2],
    'layer_3_activation': [.5, .5],
    'layer_3_dropout': [.1666, .1666, .167, .1666, .1666, .1666],
}

#return random parameter
def random_hyper_parameters(param, layer):
    hps = np.random.choice(search_space[param], p = start_probs['{}_{}' .format(layer, param)])
    return hps

In [50]:
random_hyper_parameters('nodes', 'layer_1')

512

In [51]:
def random_model_search(num_models, best_score, num):
    for j in range(0, num_models):
        model_dict = {}
        input_shape = x_train.shape[1]
        r_model = Sequential()
        input_nodes = random_hyper_parameters('nodes','input')
        input_activation = random_hyper_parameters('activation', 'input')
        input_dropout = random_hyper_parameters('dropout', 'input')
        
        model_dict['input_nodes'] = input_nodes
        model_dict['input_activation'] = input_activation
        model_dict['input_dropout'] = input_dropout
        
        r_model.add(Dense(input_nodes, input_shape = (input_shape,), activation = input_activation, activity_regularizer = l2(0.001),
               kernel_initializer = 'truncated_normal'))
        r_model.add(Dropout(input_dropout))
        
        
        hidden = random_hyper_parameters('layers', 'model')
        model_dict['model_layers'] = hidden
        
        for i in range(int(hidden)):
            layer_name = 'layer_{}' .format(i + 1)
            hidden_nodes = random_hyper_parameters('nodes', layer_name)
            hidden_activation = random_hyper_parameters('activation', layer_name)
            r_model.add(Dense(hidden_nodes, activation = hidden_activation, activity_regularizer = l2(0.001),
               kernel_initializer = 'truncated_normal'))
            dropout = random_hyper_parameters('dropout', layer_name)
            r_model.add(Dropout(dropout))
            
            model_dict['{}_nodes' .format(layer_name)] = hidden_nodes
            model_dict['{}_activation' .format(layer_name)] = hidden_activation
            model_dict['{}_dropout' .format(layer_name)] = dropout
        
        r_model.add(Dense(1, activation = 'sigmoid'))
    
        ###Create checkpoint
        r_callbacks = ModelCheckpoint(filepath = 'churn_weights_r.hdf5', monitor = 'val_loss', save_best_only = True, verbose = False)
        r_reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=20, min_lr=0.00001, verbose = 0)
        r_early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=60, restore_best_weights=True)
      
        ##compile
        optimizer = random_hyper_parameters('optimizers', 'model')
        r_model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = ['acc'] )
    
        batch_size = random_hyper_parameters('size', 'model')
        
        model_dict['model_optimizers'] = optimizer
        model_dict['model_size'] = batch_size
        ###
        history = r_model.fit(x_train, y_train, epochs = 1000, validation_split = 0.1, batch_size = batch_size, verbose = 0, callbacks = [r_callbacks, r_reduce_lr, r_early_stop])
    
    
        r_model.load_weights('churn_weights_r.hdf5')
        
        #new_preds = r_model.predict(x_test)
        #new_pred = (new_preds > 0.5).astype('int')
        #new_score = roc_auc_score(y_test, new_pred)
        
        new_score = r_model.evaluate(x_test, y_test)
        new_score = new_score[1]
        #print(new_score)
        model_dict['score'] = new_score
        
        #compare to best score, if best, store model
        #print('New Score:{}, Best Score:{}' .format(new_score, best_score))
        if new_score >= best_score:
            #print('saving model')
            r_model.save('churn_model.h5' .format(num))  
            best_score = new_score
        del r_model
        
        global new_df
        t_df = pd.DataFrame([model_dict])
        new_df = new_df.append(t_df, ignore_index = True, sort = True)
      
    return best_score

In [52]:
#sorted_df = new_df.sort_values(by = 'score')
#tail_df = sorted_df.tail(15).copy()
#tail_df.tail(15)
#sorted_df

In [53]:
def get_new_dists():
    global new_df
    
    top_num = 3#int(len(new_df)*.2)
    dists = {}
    sorted_df = new_df.sort_values(by = 'score')
    tail_df = sorted_df.tail(top_num).copy()
    
    for i in tail_df:
        if i != 'score':
            idx = i.split('_')[-1:][0]
            t_df = tail_df[i].value_counts()
            t_df = t_df.reindex(search_space[idx])
            t_df.fillna(0, inplace = True)
            values = t_df.values
            new_dist = []
            value_sum = sum(values)
            for n in range(len(values)):
                lr = .2
                new = values[n]/float(value_sum) ###HELP
                orig = start_probs[i][n]
                y = orig - ((orig-new)*lr)
                new_dist.append(y)  
            dists[i] = new_dist
    return dists
            
#start_probs = get_new_dists()

In [54]:
#start_probs

In [55]:
len(new_df)

0

In [56]:
def build_model():
    best_score = 0
    for i in range(10):
        t_score = random_model_search(10, best_score, i) ###add best score to rms
        best_score = t_score
        global start_probs
        start_probs = get_new_dists()

In [57]:
build_model()

1032/1032 [==============================] - 0s 117us/step


/Users/ericcriteser/anaconda2/envs/deeplearning/lib/python2.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


1032/1032 [==============================] - 0s 87us/step


/Users/ericcriteser/anaconda2/envs/deeplearning/lib/python2.7/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in less


1032/1032 [==============================] - 0s 119us/step


In [63]:
test_model = load_model('churn_model.h5')

In [64]:
test_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_327 (Dense)            (None, 128)               3712      
_________________________________________________________________
dropout_237 (Dropout)        (None, 128)               0         
_________________________________________________________________
dense_328 (Dense)            (None, 1024)              132096    
_________________________________________________________________
dropout_238 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_329 (Dense)            (None, 1)                 1025      
Total params: 136,833
Trainable params: 136,833
Non-trainable params: 0
_________________________________________________________________


In [65]:
score = test_model.evaluate(x_test, y_test)
score

1032/1032 [==============================] - 3s 3ms/step


[0.43818934042324392, 0.81104651162790697]

In [23]:
#new_df.tail(20)

In [66]:
new_pred = test_model.predict(x_test)
#new_pred = (new_pred > 0.5).astype('int')
new_score = roc_auc_score(y_test, new_pred)
print('Acc: {}, AUC: {}' .format(score, new_score)

0.83633569739952707

In [70]:
print('Acc: {}, AUC: {}' .format(score[1], new_score))

Acc: 0.811046511628, AUC: 0.8363356974


In [60]:
sorted_df= new_df.sort_values(by='score')
sorted_df.tail(10)

,input_activation,input_dropout,input_nodes,layer_1_activation,layer_1_dropout,layer_1_nodes,layer_2_activation,layer_2_dropout,layer_2_nodes,layer_3_activation,layer_3_dropout,layer_3_nodes,model_layers,model_optimizers,model_size,score
9,tanh,0.0,512,tanh,0.7,1024,tanh,0.5,512.0,NaN,NaN,NaN,2,sgd,128,0.809109
94,tanh,0.0,512,tanh,0.6,128,tanh,0.5,128.0,NaN,NaN,NaN,2,sgd,128,0.809109
0,relu,0.0,128,tanh,0.6,128,NaN,NaN,NaN,NaN,NaN,NaN,1,sgd,16,0.810078
35,relu,0.7,128,tanh,0.6,1024,tanh,0.5,128.0,NaN,NaN,NaN,2,adam,128,0.810078
48,tanh,0.0,128,tanh,0.7,64,tanh,0.5,128.0,NaN,NaN,NaN,2,sgd,128,0.810078
56,relu,0.5,128,tanh,0.7,128,NaN,NaN,NaN,NaN,NaN,NaN,1,adam,128,0.810078
72,tanh,0.0,128,tanh,0.6,128,tanh,0.5,128.0,NaN,NaN,NaN,2,sgd,128,0.810078
92,tanh,0.5,128,tanh,0.7,64,NaN,NaN,NaN,NaN,NaN,NaN,1,sgd,128,0.810078
30,tanh,0.5,128,tanh,0.6,128,NaN,NaN,NaN,NaN,NaN,NaN,1,sgd,128,0.811047
89,tanh,0.5,128,tanh,0.7,1024,NaN,NaN,NaN,NaN,NaN,NaN,1,sgd,128,0.811047


In [67]:
start_probs

{'input_activation': [0.28283863039999996, 0.71716136959999999],
 'input_dropout': [0.40880982305450669,
  0.017888538787839997,
  0.017931488460799998,
  0.46798347212117336,
  0.017888538787839997,
  0.069498138787839989],
 'input_nodes': [0.021474836480000006,
  0.055588509013333329,
  0.84587330901333346,
  0.055588509013333329,
  0.021474836480000006],
 'layer_1_activation': [0.053687091200000016, 0.94631290880000007],
 'layer_1_dropout': [0.017888538787839997,
  0.017888538787839997,
  0.017931488460799998,
  0.017888538787839997,
  0.60646834465450672,
  0.32193455052117337],
 'layer_1_nodes': [0.021474836480000006,
  0.13729350314666666,
  0.55844504234666659,
  0.055588509013333329,
  0.22719810901333334],
 'layer_2_activation': [nan, nan],
 'layer_2_dropout': [nan, nan, nan, nan, nan, nan],
 'layer_2_nodes': [nan, nan, nan, nan, nan],
 'layer_3_activation': [nan, nan],
 'layer_3_dropout': [nan, nan, nan, nan, nan, nan],
 'layer_3_nodes': [nan, nan, nan, nan, nan],
 'model_lay

In [34]:
#preds = model.predict(x_test)
evals = [1 if i > .15 else 0 for i in new_pred]
cf = confusion_matrix(y_test, evals)
cf

array([[32, 34],
       [ 4, 40]])

In [33]:
#total_scores = new_df['score']
#total_scores.rolling(window=3).mean().plot();